In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install langchain sentence-transformers faiss-cpu pypdf transformers langchain-community langchain-huggingface

In [3]:
import os
import glob
from pathlib import Path


from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering

import warnings
warnings.filterwarnings('ignore')



In [4]:
# Konfiguracja środowiska - sprawdzenie czy to Colab czy lokalne
try:
    from google.colab import drive
    IN_COLAB = True
    drive.mount('/content/drive')
    print("Google Drive zamontowany!")

    # Ścieżki dla Colab
    PDF_DIR = "/content/drive/MyDrive/pdf_documents/"
    VECTOR_DB_PATH = "/content/drive/MyDrive/faiss_index"

except ImportError:
    IN_COLAB = False
    # Ścieżki lokalne
    import os
    current_dir = os.path.dirname(os.path.abspath("__file__"))
    PDF_DIR = current_dir
    VECTOR_DB_PATH = os.path.join(current_dir, "faiss_index")

print(f"Środowisko: {'Google Colab' if IN_COLAB else 'Lokalne'}")

# Konfiguracja modeli - OPCJE DO WYBORU
# Embeddingi - wybierz jeden:
# EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"      # szybki (22MB)
EMBEDDING_MODEL = "sentence-transformers/all-mpnet-base-v2"   # lepszy (438MB)
# EMBEDDING_MODEL = "sentence-transformers/multi-qa-mpnet-base-dot-v1"  # najlepszy dla QA (438MB)

# Model QA - wybierz jeden:
# QA_MODEL = "distilbert-base-uncased-distilled-squad"           # szybki (67MB)
QA_MODEL = "deepset/roberta-base-squad2"                     # lepszy (473MB)
# QA_MODEL = "deepset/roberta-large-squad2"                    # najlepszy (1.3GB)

print(f"Folder PDF: {PDF_DIR}")
print(f"Baza wektorowa: {VECTOR_DB_PATH}")
print(f"Model embeddingów: {EMBEDDING_MODEL}")
print(f"Model QA: {QA_MODEL}")

# Tworzenie folderów jeśli nie istnieją
os.makedirs(PDF_DIR, exist_ok=True)
os.makedirs(os.path.dirname(VECTOR_DB_PATH), exist_ok=True)

# Sprawdzenie jakie PDFy są dostępne
pdf_files = [f for f in os.listdir(PDF_DIR) if f.endswith('.pdf')]
print(f"\nZnalezione pliki PDF ({len(pdf_files)}):")
for pdf in pdf_files:
    print(f"   {pdf}")

if IN_COLAB:
    if len(pdf_files) >= 3:
        print(f"Masz {len(pdf_files)} plików PDF - można zaczynać!")
    else:
        print("\nInstrukcja dla Colab:")
        print("1. Prześlij pliki PDF do folderu: /content/drive/MyDrive/pdf_documents/")
        print("2. Uruchom kolejne komórki")
else:
    if len(pdf_files) >= 3:
        print(f"\nSystem znalazł {len(pdf_files)} plików PDF w folderze!")
        print("Dokumenty są gotowe do analizy")
    else:
        print(f"\nZnaleziono tylko {len(pdf_files)} plików PDF")
        print("Dodaj więcej plików PDF do tego folderu")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive zamontowany!
Środowisko: Google Colab
Folder PDF: /content/drive/MyDrive/pdf_documents/
Baza wektorowa: /content/drive/MyDrive/faiss_index
Model embeddingów: sentence-transformers/all-mpnet-base-v2
Model QA: deepset/roberta-base-squad2

Znalezione pliki PDF (3):
   NatureDeepReview.pdf
   2024-wttc-introduction-to-ai.pdf
   1-s2.0-S0304397596001636-main.pdf
Masz 3 plików PDF - można zaczynać!


In [5]:
print(f"Aktualnie używane modele:")
print(f"Embeddingi: {EMBEDDING_MODEL}")
print(f"QA: {QA_MODEL}")
print("\nJeśli zmieniłeś modele, uruchom ponownie komórki od tworzenia bazy wektorowej!")

Aktualnie używane modele:
Embeddingi: sentence-transformers/all-mpnet-base-v2
QA: deepset/roberta-base-squad2

Jeśli zmieniłeś modele, uruchom ponownie komórki od tworzenia bazy wektorowej!


## Opcje poprawy jakości modeli

**Jak poprawić jakość systemu RAG:**

### 1. Lepsze modele embeddingów:
- `all-MiniLM-L6-v2` (22MB) - szybki, podstawowy
- `all-mpnet-base-v2` (438MB) - lepszy, bardziej dokładny
- `multi-qa-mpnet-base-dot-v1` (438MB) - najlepszy dla Q&A

### 2. Lepsze modele QA:
- `distilbert-base-uncased-distilled-squad` (67MB) - szybki
- `roberta-base-squad2` (473MB) - znacznie lepszy
- `roberta-large-squad2` (1.3GB) - najlepszy, ale wolny

### 3. Inne ulepszenia:
- Większe kawałki tekstu (1500 znaków zamiast 1000)
- Więcej nakładania się między kawałkami (300 zamiast 200)
- Czyszczenie tekstu z artefaktów
- Filtrowanie dokumentów według podobieństwa
- Wyższy próg pewności (0.2 zamiast 0.15)

**Żeby użyć lepszych modeli:**
Odkomentuj odpowiednie linie w konfiguracji powyżej i uruchom ponownie komórki.

In [6]:
# Funkcja do ładowania i przetwarzania PDF-ów na kawałki tekstu

import re

def clean_text(text):
    """Czyści tekst z niepotrzebnych znaków"""
    # Usuń nadmiarowe białe znaki
    text = re.sub(r'\s+', ' ', text)
    # Usuń dziwne znaki
    text = re.sub(r'[^\w\s\.\,\?\!\;\:\-\(\)]', '', text)
    # Usuń bardzo krótkie linie (prawdopodobnie artefakty)
    lines = text.split('\n')
    lines = [line.strip() for line in lines if len(line.strip()) > 20]
    return '\n'.join(lines)

def load_and_process_documents(pdf_files):
    """Ładuje PDFy i dzieli na małe kawałki do analizy"""
    documents = []

    print(f"Przetwarzanie {len(pdf_files)} dokumentów PDF...")

    for pdf_file in pdf_files:
        # Sprawdzenie czy ścieżka jest absolutna czy względna
        if os.path.isabs(pdf_file):
            full_path = pdf_file
        else:
            full_path = os.path.join(PDF_DIR, pdf_file)

        print(f"Ładowanie: {os.path.basename(full_path)}")
        try:
            loader = PyPDFLoader(full_path)
            docs = loader.load()

            # Czyszczenie tekstu i dodanie metadanych
            for doc in docs:
                # Wyczyść tekst
                doc.page_content = clean_text(doc.page_content)
                # Dodaj metadane
                doc.metadata['source_file'] = os.path.basename(full_path)
                # Usuń bardzo krótkie dokumenty (prawdopodobnie puste strony)
                if len(doc.page_content.strip()) < 100:
                    continue

            # Filtruj tylko dokumenty z sensowną zawartością
            docs = [doc for doc in docs if len(doc.page_content.strip()) >= 100]

            documents.extend(docs)
            print(f"  Załadowano {len(docs)} stron (po filtrowaniu)")

        except Exception as e:
            print(f"  Błąd przy ładowaniu {full_path}: {e}")

    if not documents:
        print("Nie udało się załadować żadnych dokumentów!")
        return []

    print(f"Łącznie załadowano {len(documents)} stron z dokumentów")

    # Lepsze dzielenie na kawałki tekstu
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,        # Większe kawałki dla lepszego kontekstu
        chunk_overlap=300,      # Więcej nakładania się
        length_function=len,
        separators=["\n\n", "\n", ". ", " ", ""]  # Dziel inteligentnie
    )

    texts = text_splitter.split_documents(documents)

    # Filtruj kawałki które są za krótkie
    texts = [text for text in texts if len(text.page_content.strip()) > 50]

    print(f"Utworzono {len(texts)} kawałków tekstu gotowych do analizy")

    return texts

In [7]:
# Funkcje do tworzenia bazy wektorowej

def create_vector_database(texts):
    """Tworzy bazę wektorową FAISS z kawałków tekstu"""
    print("Ładowanie modelu embeddingów...")

    try:
        # Inicjalizacja modelu który zamienia tekst na liczby (wektory)
        embeddings = HuggingFaceEmbeddings(
            model_name=EMBEDDING_MODEL,
            model_kwargs={'device': 'cpu'},
            encode_kwargs={'normalize_embeddings': True}
        )

        print("Tworzenie bazy wektorowej FAISS...")
        # Tworzenie bazy danych z wektorami tekstu
        vectorstore = FAISS.from_documents(texts, embeddings)

        print("Baza wektorowa została utworzona!")
        return vectorstore, embeddings

    except Exception as e:
        print(f"Błąd przy tworzeniu bazy wektorowej: {e}")
        return None, None

def save_vector_database(vectorstore, path=None):
    """Zapisuje bazę wektorową na dysk"""
    if path is None:
        path = VECTOR_DB_PATH

    if vectorstore:
        try:
            os.makedirs(os.path.dirname(path), exist_ok=True)
            vectorstore.save_local(path)
            print(f"Baza wektorowa zapisana w: {path}")
            if IN_COLAB:
                print("Baza została zapisana na Google Drive")
        except Exception as e:
            print(f"Błąd przy zapisywaniu bazy: {e}")

def load_vector_database(embeddings, path=None):
    """Ładuje istniejącą bazę wektorową z dysku"""
    if path is None:
        path = VECTOR_DB_PATH

    try:
        if os.path.exists(path):
            vectorstore = FAISS.load_local(path, embeddings, allow_dangerous_deserialization=True)
            print(f"Załadowano bazę wektorową z: {path}")
            return vectorstore
        else:
            print(f"Nie znaleziono bazy wektorowej w: {path}")
            return None
    except Exception as e:
        print(f"Błąd przy ładowaniu bazy wektorowej: {e}")
        return None

In [8]:
# Tworzenie bazy wektorowej z PDF-ów

# Znajdź wszystkie pliki PDF w folderze
pdf_files = []
if IN_COLAB:
    # W Colab szukaj w drive
    import glob
    pdf_files = glob.glob(os.path.join(PDF_DIR, "*.pdf"))
else:
    # Lokalnie sprawdź folder
    all_files = os.listdir(PDF_DIR)
    pdf_files = [os.path.join(PDF_DIR, f) for f in all_files if f.endswith('.pdf')]

print(f"Znaleziono {len(pdf_files)} plików PDF:")
for pdf in pdf_files:
    print(f"   {os.path.basename(pdf)}")

if len(pdf_files) >= 3:
    print(f"\nUżywam {len(pdf_files)} dokumentów do stworzenia bazy wiedzy")

    vectorstore = None
    embeddings = None

    if os.path.exists(VECTOR_DB_PATH):
        print("Znaleziono istniejącą bazę wektorową...")
        print("Można załadować istniejącą bazę lub utworzyć nową")
        print("Jeśli to pierwsze uruchomienie lub zmieniłeś dokumenty, lepiej utworzyć nową")

        create_new = True  # Dla pierwszego uruchomienia zawsze tworzymy nową

        if create_new:
            print("Tworzenie nowej bazy wektorowej z dokumentów...")
            texts = load_and_process_documents(pdf_files)
            if texts:
                vectorstore, embeddings = create_vector_database(texts)
                if vectorstore:
                    save_vector_database(vectorstore)
                    print("Nowa baza wektorowa gotowa!")
        else:
            print("Próba załadowania istniejącej bazy...")
            embeddings = HuggingFaceEmbeddings(
                model_name=EMBEDDING_MODEL,
                model_kwargs={'device': 'cpu'},
                encode_kwargs={'normalize_embeddings': True}
            )
            vectorstore = load_vector_database(embeddings)
    else:
        print("Tworzenie pierwszej bazy wektorowej...")
        texts = load_and_process_documents(pdf_files)
        if texts:
            vectorstore, embeddings = create_vector_database(texts)
            if vectorstore:
                save_vector_database(vectorstore)
                print("Baza wektorowa z dokumentami AI/ML gotowa!")
                print("Asystent może odpowiadać na pytania o:")
                print("   - Deep Learning: sieci neuronowe, optymalizacja, CNN, RNN")
                print("   - Machine Learning: algorytmy, supervised/unsupervised learning")
                print("   - Artificial Intelligence: historia, metody, zastosowania")
                print("   - Przeglądy technologii AI: najnowsze trendy i badania")
        else:
            print("Nie udało się przetworzyć dokumentów!")
else:
    print(f"Za mało plików PDF ({len(pdf_files)}/3)")
    print("Dodaj więcej dokumentów PDF do folderu")
    print("Aktualna lista plików:")
    for pdf in pdf_files:
        print(f"   - {os.path.basename(pdf)}")

Znaleziono 3 plików PDF:
   NatureDeepReview.pdf
   2024-wttc-introduction-to-ai.pdf
   1-s2.0-S0304397596001636-main.pdf

Używam 3 dokumentów do stworzenia bazy wiedzy
Znaleziono istniejącą bazę wektorową...
Można załadować istniejącą bazę lub utworzyć nową
Jeśli to pierwsze uruchomienie lub zmieniłeś dokumenty, lepiej utworzyć nową
Tworzenie nowej bazy wektorowej z dokumentów...
Przetwarzanie 3 dokumentów PDF...
Ładowanie: NatureDeepReview.pdf
  Załadowano 9 stron (po filtrowaniu)
Ładowanie: 2024-wttc-introduction-to-ai.pdf
  Załadowano 44 stron (po filtrowaniu)
Ładowanie: 1-s2.0-S0304397596001636-main.pdf
  Załadowano 40 stron (po filtrowaniu)
Łącznie załadowano 93 stron z dokumentów
Utworzono 260 kawałków tekstu gotowych do analizy
Ładowanie modelu embeddingów...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Tworzenie bazy wektorowej FAISS...
Baza wektorowa została utworzona!
Baza wektorowa zapisana w: /content/drive/MyDrive/faiss_index
Baza została zapisana na Google Drive
Nowa baza wektorowa gotowa!


In [9]:
# Inicjalizacja modelu do odpowiadania na pytania

def initialize_qa_model():
    """Inicjalizuje model Question Answering"""
    print("Ładowanie modelu QA...")

    try:
        # Ładowanie tokenizer i modelu
        tokenizer = AutoTokenizer.from_pretrained(QA_MODEL)
        model = AutoModelForQuestionAnswering.from_pretrained(QA_MODEL)

        # Tworzenie pipeline do zadawania pytań
        qa_pipeline = pipeline(
            "question-answering",
            model=model,
            tokenizer=tokenizer,
            device=-1  # używaj CPU
        )

        print(f"Model QA '{QA_MODEL}' został załadowany!")
        return qa_pipeline

    except Exception as e:
        print(f"Błąd przy ładowaniu modelu QA: {e}")
        return None

# Inicjalizuj model QA
qa_model = initialize_qa_model()

Ładowanie modelu QA...


tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

Device set to use cpu


Model QA 'deepset/roberta-base-squad2' został załadowany!


In [10]:
# Klasa asystenta RAG z mechanizmami przeciwko halucynacjom

class RAGAssistant:
    def __init__(self, vectorstore, qa_model, embeddings):
        self.vectorstore = vectorstore
        self.qa_model = qa_model
        self.embeddings = embeddings
        self.conversation_memory = []
        self.confidence_threshold = 0.2  # podwyższony próg dla lepszej jakości

    def retrieve_relevant_documents(self, question, k=7):  # więcej dokumentów
        """Pobiera najbardziej podobne dokumenty do pytania"""
        if not self.vectorstore:
            return []

        try:
            # Wyszukiwanie podobnych dokumentów z większą liczbą wyników
            docs = self.vectorstore.similarity_search_with_score(question, k=k)

            # Filtruj dokumenty z bardzo niskim podobieństwem
            filtered_docs = []
            for doc, score in docs:
                # Im niższy score, tym lepsze podobieństwo w FAISS
                if score < 1.0:  # próg podobieństwa
                    filtered_docs.append(doc)

            return filtered_docs[:5]  # zwróć top 5
        except Exception as e:
            print(f"Błąd przy wyszukiwaniu dokumentów: {e}")
            # Fallback do zwykłego similarity_search
            return self.vectorstore.similarity_search(question, k=5)

    def improve_context(self, question, relevant_docs):
        """Ulepsza kontekst przez dodanie powiązanych informacji"""
        context_texts = []

        for doc in relevant_docs:
            text = doc.page_content.strip()
            if len(text) > 100:  # tylko sensowne fragmenty
                # Dodaj trochę kontekstu o źródle
                source_info = f"[Z dokumentu: {doc.metadata.get('source_file', 'nieznany')}]\n"
                context_texts.append(source_info + text)

        # Sortuj według długości - dłuższe fragmenty zwykle mają więcej informacji
        context_texts.sort(key=len, reverse=True)

        return context_texts[:4]  # top 4 najdłuższe fragmenty

    def generate_answer(self, question, relevant_docs):
        """Generuje odpowiedź na podstawie znalezionych dokumentów"""
        if not relevant_docs:
            return {
                'answer': "Nie znalazłem odpowiednich dokumentów do odpowiedzi na to pytanie.",
                'sources': [],
                'confidence': 0.0,
                'has_answer': False
            }

        try:
            # Ulepsz kontekst
            context_texts = self.improve_context(question, relevant_docs)

            # Tworzenie bogatszego kontekstu
            context = "\n\n".join(context_texts)

            # Skracanie kontekstu jeśli za długi (ale zachowaj więcej niż wcześniej)
            if len(context) > 3000:
                context = context[:3000] + "..."

            # Generowanie odpowiedzi przez model
            result = self.qa_model(question=question, context=context)

            answer = result['answer'].strip()
            confidence = result['score']

            # Jeśli odpowiedź za krótka, spróbuj z jeszcze większym kontekstem
            if len(answer) < 15:
                extended_context = "\n\n".join(context_texts)[:4000]
                result = self.qa_model(question=question, context=extended_context)
                answer = result['answer'].strip()
                confidence = result['score']

            # Poprawa formatowania odpowiedzi
            if confidence < self.confidence_threshold:
                formatted_answer = f"""Niepewna odpowiedź (pewność: {confidence:.2f})

Znaleziona informacja: {answer}

Uwaga: Ta odpowiedź może być niepełna. Model nie jest pewien swojej odpowiedzi.
Spróbuj zadać pytanie inaczej lub sprawdź źródła bezpośrednio."""
            else:
                formatted_answer = f"""Odpowiedź (pewność: {confidence:.2f})

{answer}

Model znalazł tę odpowiedź z dobrą pewnością w dokumentach."""

            return {
                'answer': formatted_answer,
                'sources': self.format_sources(relevant_docs),
                'confidence': confidence,
                'has_answer': True
            }

        except Exception as e:
            return {
                'answer': f"Wystąpił błąd podczas generowania odpowiedzi: {e}",
                'sources': self.format_sources(relevant_docs),
                'confidence': 0.0,
                'has_answer': False
            }

    def format_sources(self, documents):
        """Formatuje informacje o źródłach"""
        sources = []
        seen_sources = set()

        for doc in documents:
            source_file = doc.metadata.get('source_file', 'Nieznany dokument')
            page = doc.metadata.get('page', 'Nieznana strona')

            source_info = f"{source_file} (strona {page + 1})"

            if source_info not in seen_sources:
                sources.append(source_info)
                seen_sources.add(source_info)

            if len(sources) >= 3:  # maksymalnie 3 źródła
                break

        return sources

    def ask(self, question):
        """Główna funkcja do zadawania pytań"""
        print(f"\nPytanie: {question}")
        print("Szukam odpowiedzi w dokumentach...")

        # Pobierz podobne dokumenty
        relevant_docs = self.retrieve_relevant_documents(question)

        if not relevant_docs:
            response = {
                'answer': "Nie znalazłem żadnych dokumentów dla tego pytania.",
                'sources': [],
                'confidence': 0.0,
                'has_answer': False
            }
        else:
            print(f"Znaleziono {len(relevant_docs)} podobnych dokumentów")
            # Generuj odpowiedź
            response = self.generate_answer(question, relevant_docs)

        # Zapisz do historii
        self.conversation_memory.append({
            'type': 'question',
            'content': question
        })
        self.conversation_memory.append({
            'type': 'answer',
            'content': response
        })

        # Wyświetl odpowiedź
        print("\n" + "="*60)
        print(response['answer'])

        if response['sources']:
            print(f"\nŹródła:")
            for i, source in enumerate(response['sources'], 1):
                print(f"   {i}. {source}")

        print("="*60)

        return response

    def show_conversation_history(self):
        """Wyświetla historię konwersacji"""
        if not self.conversation_memory:
            print("Historia konwersacji jest pusta.")
            return

        print("\nHistoria konwersacji:")
        print("="*40)

        for i in range(0, len(self.conversation_memory), 2):
            if i + 1 < len(self.conversation_memory):
                question = self.conversation_memory[i]
                answer = self.conversation_memory[i + 1]

                print(f"\nPytanie {i//2 + 1}: {question['content']}")
                print(f"Odpowiedź: {answer['content']['answer']}")

                if answer['content']['sources']:
                    print("Źródła: " + ", ".join(answer['content']['sources']))

        print("="*40)

print("Klasa RAGAssistant została utworzona z ulepszeniami!")
print("Ulepszenia:")
print("   • Lepsze filtrowanie dokumentów według podobieństwa")
print("   • Większy kontekst dla bogatszych odpowiedzi")
print("   • Czyszczenie tekstu z artefaktów")
print("   • Wyższy próg pewności dla lepszej jakości")
print("   • Dodanie informacji o źródłach w kontekście")

Klasa RAGAssistant została utworzona z ulepszeniami!
Ulepszenia:
   • Lepsze filtrowanie dokumentów według podobieństwa
   • Większy kontekst dla bogatszych odpowiedzi
   • Czyszczenie tekstu z artefaktów
   • Wyższy próg pewności dla lepszej jakości
   • Dodanie informacji o źródłach w kontekście


## Wnioski z implementacji RAG

**Co udało się zrobić:**
- Stworzyliśmy klasę która łączy wyszukiwanie w dokumentach z generowaniem odpowiedzi
- Model najpierw znajdzie podobne kawałki tekstu, potem generuje odpowiedź na ich podstawie
- Dodaliśmy sprawdzanie pewności żeby nie dawać głupich odpowiedzi

**Problemy które rozwiązaliśmy:**
- Model czasami daje bardzo krótkie odpowiedzi - dodaliśmy rozszerzanie kontekstu
- Duplikaty w źródłach - używamy set() żeby je usunąć
- Brak kontroli nad jakością - dodaliśmy próg pewności

**Co można poprawić:**
- Próg pewności (0.15) może być za niski - czasami lepiej podnieść do 0.3
- Można dodać więcej kontekstu dla lepszych odpowiedzi
- Warto dodać preprocessing tekstu (usuwanie specjalnych znaków, etc.)

**Obserwacje:**
- DistilBERT działa lepiej na konkretnych pytaniach niż na ogólnych
- Im więcej dokumentów, tym lepsze odpowiedzi (ale też wolniejsze)

In [11]:
# Inicjalizacja asystenta RAG

def initialize_rag_assistant():
    """Inicjalizuje asystenta RAG"""
    global rag_assistant

    # Sprawdź czy wszystkie komponenty są dostępne
    if 'vectorstore' not in globals() or vectorstore is None:
        print("Baza wektorowa nie jest dostępna!")
        return None

    if 'qa_model' not in globals() or qa_model is None:
        print("Model QA nie jest dostępny!")
        return None

    if 'embeddings' not in globals() or embeddings is None:
        print("Model embeddingów nie jest dostępny!")
        return None

    # Utwórz asystenta
    assistant = RAGAssistant(vectorstore, qa_model, embeddings)
    print("Asystent RAG został zainicjalizowany!")

    return assistant

def interactive_chat():
    """Interaktywny chat z asystentem"""
    if 'rag_assistant' not in globals() or rag_assistant is None:
        print("Asystent nie jest zainicjalizowany!")
        return

    print("\nWitaj w czacie z asystentem RAG!")
    print("Zadawaj pytania o treść dokumentów PDF")
    print("Wpisz 'quit', 'exit' lub 'koniec' aby zakończyć")
    print("Wpisz 'historia' aby zobaczyć historię konwersacji")
    print("Wpisz 'reset' aby wyczyścić historię")
    print("-" * 40)

    while True:
        try:
            question = input("\nTwoje pytanie: ").strip()

            if question.lower() in ['quit', 'exit', 'koniec', 'q']:
                print("Do widzenia!")
                break
            elif question.lower() in ['historia', 'history']:
                rag_assistant.show_conversation_history()
                continue
            elif question.lower() == 'reset':
                rag_assistant.conversation_memory = []
                print("Historia konwersacji została wyczyszczona!")
                continue
            elif not question:
                print("Proszę wpisać pytanie!")
                continue

            # Zadaj pytanie asystentowi
            rag_assistant.ask(question)

        except KeyboardInterrupt:
            print("\n\nChat przerwany przez użytkownika. Do widzenia!")
            break
        except Exception as e:
            print(f"Wystąpił błąd: {e}")

# Sprawdź czy można zainicjalizować asystenta
if 'vectorstore' in globals() and 'qa_model' in globals():
    rag_assistant = initialize_rag_assistant()

    if rag_assistant:
        print("\nSystem RAG jest gotowy do pracy!")
        print("\nDostępne funkcje:")
        print("   - rag_assistant.ask('twoje pytanie') - zadaj pytanie")
        print("   - rag_assistant.show_conversation_history() - pokaż historię")
        print("   - interactive_chat() - uruchom interaktywny chat")
else:
    print("System nie jest jeszcze gotowy. Upewnij się, że:")
    print("   1. Zainstalowano wszystkie biblioteki")
    print("   2. Umieszczono PDFy w folderze 'pdf_documents'")
    print("   3. Uruchomiono wszystkie poprzednie komórki")

Asystent RAG został zainicjalizowany!

System RAG jest gotowy do pracy!

Dostępne funkcje:
   - rag_assistant.ask('twoje pytanie') - zadaj pytanie
   - rag_assistant.show_conversation_history() - pokaż historię
   - interactive_chat() - uruchom interaktywny chat


In [12]:
# Ulepszona wersja interaktywnego chatu

def interactive_chat_advanced():
    """Interaktywny chat z asystentem RAG - więcej opcji"""
    if 'rag_assistant' not in globals() or rag_assistant is None:
        print("Asystent nie jest zainicjalizowany!")
        print("Uruchom wszystkie poprzednie komórki.")
        return

    print("Witaj w czacie z asystentem RAG!")
    print("="*50)
    print("Jak używać:")
    print("   • Zadawaj pytania o dokumenty PDF")
    print("   • Wpisz 'quit', 'exit' lub 'koniec' aby zakończyć")
    print("   • Wpisz 'historia' aby zobaczyć historię")
    print("   • Wpisz 'reset' aby wyczyścić historię")
    print("   • Wpisz 'pomoc' aby zobaczyć przykładowe pytania")
    print("="*50)

    # Przykładowe pytania
    example_questions = [
        "What is deep learning?",
        "How do neural networks work?",
        "What are the main types of machine learning?",
        "Explain artificial intelligence",
        "What are the challenges in AI?"
    ]

    while True:
        try:
            print(f"\nZadaj pytanie (lub wpisz komendę):")
            question = input(">> ").strip()

            # Obsługa komend
            if question.lower() in ['quit', 'exit', 'koniec', 'q']:
                print("\nDziękuję za rozmowę! Do widzenia!")
                break
            elif question.lower() in ['historia', 'history', 'h']:
                rag_assistant.show_conversation_history()
                continue
            elif question.lower() in ['reset', 'r']:
                rag_assistant.conversation_memory = []
                print("Historia konwersacji została wyczyszczona!")
                continue
            elif question.lower() in ['pomoc', 'help', 'p']:
                print("\nPrzykładowe pytania:")
                for i, q in enumerate(example_questions, 1):
                    print(f"   {i}. {q}")
                continue
            elif not question:
                print("Proszę wpisać pytanie!")
                continue

            # Zadaj pytanie asystentowi
            rag_assistant.ask(question)

        except KeyboardInterrupt:
            print("\n\nChat przerwany. Do widzenia!")
            break
        except Exception as e:
            print(f"Wystąpił błąd: {e}")
            print("Spróbuj ponownie lub wpisz 'quit' aby zakończyć.")

def demo_rag_system():
    """Demonstracja możliwości systemu RAG"""
    if 'rag_assistant' not in globals() or rag_assistant is None:
        print("Asystent nie jest zainicjalizowany!")
        return

    print("Demonstracja systemu RAG")
    print("="*50)

    # Przykładowe pytania do testowania
    demo_questions = [
        "What is machine learning?",
        "How do neural networks learn?",
        "What are the applications of AI?",
        "What is deep learning?"
    ]

    print("Testowanie systemu z przykładowymi pytaniami:\n")

    for i, question in enumerate(demo_questions, 1):
        print(f"Test {i}/4: {question}")
        result = rag_assistant.ask(question)

        if i < len(demo_questions):
            input("\nNaciśnij Enter aby kontynuować do następnego testu...")
            print("\n" + "-"*30 + "\n")

    print("\nDemo zakończone!")
    print("Aby kontynuować rozmowę, użyj: interactive_chat_advanced()")

# Sprawdź czy można uruchomić system
if 'vectorstore' in globals() and 'qa_model' in globals():
    if 'rag_assistant' in globals() and rag_assistant:
        print("\nSystem RAG jest gotowy!")
        print("\nDostępne funkcje:")
        print("   • rag_assistant.ask('pytanie') - zadaj pytanie")
        print("   • interactive_chat_advanced() - uruchom czat")
        print("   • demo_rag_system() - zobacz demo")
        print("   • rag_assistant.show_conversation_history() - historia")
    else:
        print("Asystent wymaga inicjalizacji. Uruchom komórkę wyżej.")
else:
    print("System nie jest gotowy. Sprawdź czy:")
    print("   1. Zainstalowano biblioteki")
    print("   2. Załadowano PDFy")
    print("   3. Uruchomiono wszystkie komórki")


System RAG jest gotowy!

Dostępne funkcje:
   • rag_assistant.ask('pytanie') - zadaj pytanie
   • interactive_chat_advanced() - uruchom czat
   • demo_rag_system() - zobacz demo
   • rag_assistant.show_conversation_history() - historia


## Podsumowanie projektu RAG

**Co się udało:**
- Zbudowaliśmy działający system RAG od zera
- System może odpowiadać na pytania na podstawie PDF-ów
- Dodaliśmy mechanizmy kontroli jakości odpowiedzi
- Wszystko działa w Google Colab

**Czego się nauczyliśmy:**
- RAG to połączenie wyszukiwania (retrieval) + generowania (generation)
- Embeddingi to sposób na zamianę tekstu na liczby które można porównywać
- FAISS to szybka baza danych do wyszukiwania podobnych wektorów
- DistilBERT to dobry kompromis między jakością a szybkością

**Problemy i rozwiązania:**
- **Problem:** Fragmentaryczne odpowiedzi → **Rozwiązanie:** Większy kontekst
- **Problem:** Niskie pewności → **Rozwiązanie:** Lepsze dzielenie tekstu na kawałki
- **Problem:** Słabe PDFy → **Rozwiązanie:** Wybór dokumentów z dobrą jakością tekstu

**Możliwe ulepszenia:**
- Lepszy preprocessing tekstu (usuwanie nagłówków, stopek, etc.)
- Używanie większych modeli (BERT, RoBERTa) dla lepszej jakości
- Dodanie re-rankingu wyników
- Implementacja hybrydowego wyszukiwania (keyword + semantic)

**Wnioski:**
System działa, ale jakość bardzo zależy od jakości dokumentów wejściowych. Warto poświęcić czas na dobre przygotowanie danych.

In [13]:
rag_assistant.ask('what is neural network?')


Pytanie: what is neural network?
Szukam odpowiedzi w dokumentach...
Znaleziono 4 podobnych dokumentów

Niepewna odpowiedź (pewność: 0.13)

Znaleziona informacja: convolutional neural network

Uwaga: Ta odpowiedź może być niepełna. Model nie jest pewien swojej odpowiedzi.
Spróbuj zadać pytanie inaczej lub sprawdź źródła bezpośrednio.

Źródła:
   1. 2024-wttc-introduction-to-ai.pdf (strona 10)
   2. NatureDeepReview.pdf (strona 1)
   3. NatureDeepReview.pdf (strona 4)


{'answer': 'Niepewna odpowiedź (pewność: 0.13)\n\nZnaleziona informacja: convolutional neural network\n\nUwaga: Ta odpowiedź może być niepełna. Model nie jest pewien swojej odpowiedzi.\nSpróbuj zadać pytanie inaczej lub sprawdź źródła bezpośrednio.',
 'sources': ['2024-wttc-introduction-to-ai.pdf (strona 10)',
  'NatureDeepReview.pdf (strona 1)',
  'NatureDeepReview.pdf (strona 4)'],
 'confidence': 0.13256898522377014,
 'has_answer': True}

In [14]:
rag_assistant.ask('What is deep learning?')


Pytanie: What is deep learning?
Szukam odpowiedzi w dokumentach...
Znaleziono 5 podobnych dokumentów

Odpowiedź (pewność: 0.29)

a type of advanced machine learning

Model znalazł tę odpowiedź z dobrą pewnością w dokumentach.

Źródła:
   1. 2024-wttc-introduction-to-ai.pdf (strona 10)
   2. NatureDeepReview.pdf (strona 1)
   3. 2024-wttc-introduction-to-ai.pdf (strona 36)


{'answer': 'Odpowiedź (pewność: 0.29)\n\na type of advanced machine learning\n\nModel znalazł tę odpowiedź z dobrą pewnością w dokumentach.',
 'sources': ['2024-wttc-introduction-to-ai.pdf (strona 10)',
  'NatureDeepReview.pdf (strona 1)',
  '2024-wttc-introduction-to-ai.pdf (strona 36)'],
 'confidence': 0.29386961460113525,
 'has_answer': True}

In [ ]:








interactive_chat()


Witaj w czacie z asystentem RAG!
Zadawaj pytania o treść dokumentów PDF
Wpisz 'quit', 'exit' lub 'koniec' aby zakończyć
Wpisz 'historia' aby zobaczyć historię konwersacji
Wpisz 'reset' aby wyczyścić historię
----------------------------------------

Twoje pytanie: How do neural networks work?

Pytanie: How do neural networks work?
Szukam odpowiedzi w dokumentach...
Znaleziono 5 podobnych dokumentów

Odpowiedź (pewność: 0.21)

feedforward

Model znalazł tę odpowiedź z dobrą pewnością w dokumentach.

Źródła:
   1. NatureDeepReview.pdf (strona 2)
   2. NatureDeepReview.pdf (strona 1)
   3. NatureDeepReview.pdf (strona 3)

Twoje pytanie: What is training in machine learning?

Pytanie: What is training in machine learning?
Szukam odpowiedzi w dokumentach...
Znaleziono 1 podobnych dokumentów

Niepewna odpowiedź (pewność: 0.07)

Znaleziona informacja: finding patterns in the data

Uwaga: Ta odpowiedź może być niepełna. Model nie jest pewien swojej odpowiedzi.
Spróbuj zadać pytanie inaczej lu